## Import relevant packages

In [56]:
import pandas as pd
import os
import chardet
import ast
import json

### Check the working directory

In [20]:
pwd

'C:\\Users\\Hwai Teng Teoh'

### Import Business.csv file

In [27]:
with open(r"C:\Users\Hwai Teng Teoh\Documents\Yelp Data\restaurant.csv", 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large


business=pd.read_csv(r"C:\Users\Hwai Teng Teoh\Documents\Yelp Data\restaurant.csv", encoding=result['encoding'])
business.head()

,name,business_id,address,review_count,stars,categories,longitude,latitude,city,DietaryRestrictions,WiFi,GoodForKids,RestaurantsTakeOut,RestaurantsReservations,Smoking,MealType
0,The Cold Pressery,1nkgeDBTswaYaNm_SSGC8A,"4310 Sherwoodtowne Boulevard, Unit 5",155,4.0,Juice Bars & Smoothies,-79.641877,43.599632,MISSISSAUGA,vegan,True,False,False,False,Restricted,dessert
1,The Cold Pressery,1nkgeDBTswaYaNm_SSGC8A,"4310 Sherwoodtowne Boulevard, Unit 5",155,4.0,Cafes,-79.641877,43.599632,MISSISSAUGA,vegan,True,False,False,False,Restricted,dessert
2,The Cold Pressery,1nkgeDBTswaYaNm_SSGC8A,"4310 Sherwoodtowne Boulevard, Unit 5",155,4.0,Vegan,-79.641877,43.599632,MISSISSAUGA,vegan,True,False,False,False,Restricted,dessert
3,Sugar Marmalade,2PCz_uVX7GOXtGHNXAPXhw,"3278 Midland Avenue, Unit D106",70,3.0,Desserts,-79.288043,43.804929,TORONTO,general,True,False,False,False,Restricted,dessert
4,Sugar Marmalade,2PCz_uVX7GOXtGHNXAPXhw,"3278 Midland Avenue, Unit D106",70,3.0,Juice Bars & Smoothies,-79.288043,43.804929,TORONTO,general,True,False,False,False,Restricted,dessert


### Import Review.json file

#### Method 1: Read JSON as strings

In [28]:
review_json_path ='Documents/Yelp Data/yelp_academic_dataset_review.json'

In [36]:
with open(review_json_path, encoding='utf-8') as json_file:      
    data = json_file.readlines()
    # this line below may take at least 8-10 minutes of processing for 4-5 million rows. It converts all strings in list to actual json objects. 
    data = list(map(json.loads, data)) 

In [37]:
# Convert to Data Frame
review=pd.DataFrame(data)

# Check 
type(review)

In [44]:
# Check first 5 observations
review.head()

,0
0,"{""review_id"":""xQY8N_XvtGbearJ5X4QryQ"",""user_id..."
1,"{""review_id"":""UmFMZ8PyXZTY2QcwzsfQYA"",""user_id..."
2,"{""review_id"":""LG2ZaYiOgpr2DK_90pYjNw"",""user_id..."
3,"{""review_id"":""i6g_oA9Yf9Y31qt0wibXpw"",""user_id..."
4,"{""review_id"":""6TdNDKywdbjoTkizeMce8A"",""user_id..."


In [45]:
# Write to CSV
csv_name = "yelp_reviews.csv"
review.to_csv(csv_name, index=False)

#### Method 2 (Chosen)

#### Get the business ids from the cleansed business data frame

In [47]:
print('Getting business_ids')
business_ids = pd.Series(business['business_id'].index.values, index=business['business_id']).to_dict()

Getting business_ids


In [64]:
type(business_ids)

dict

#### Only read the review data those business ids for the businesses

In [57]:
# REVIEW.JSON
review_data = []
print('Reading review.json')
with open(review_json_path, encoding='utf-8') as f:
    for line in f:
        newline = ast.literal_eval(line)  # read the line (str) as a dict
        if newline['business_id'] in business_ids:
            review_data.append(json.loads(line))
review_df = pd.DataFrame.from_dict(review_data)
# get list of business id's from business.json and only keep reviews of those businesses
# would normally write it with to_json but it's too big to fit in memory, so write it line by line

Reading review.json


In [60]:
# Drop irrelevant columns
review_df=review_df.drop(['review_id','user_id'], axis=1)

In [61]:
# Check first 5 observations 
review_df.head()

,business_id,stars,useful,funny,cool,text,date
0,l-nL4BmhzpZjcavooO48PQ,4.0,2,1,1,ORDER In (Delivery) Review\n\nI discovered thi...,2015-10-17 01:38:13
1,qUWqjjjfpB2-4P3He5rsKw,4.0,0,0,0,"In the heart of Chinatown, I discovered it enr...",2017-06-06 19:04:10
2,YJ2Y_asLIlbo-uijVugLow,5.0,0,0,2,"As a spicy food fan, when I see a restaurant c...",2015-06-08 18:24:07
3,9Eghhu_LzEJgDKNgisf3rg,3.0,0,0,0,Was super excited to bring my boyfriend to thi...,2016-08-25 16:25:12
4,eNFubUPJR7yIQah-NxZN_g,2.0,0,0,0,Decent food. Fishermen lobster or even congee ...,2017-05-26 01:20:43


#### Write to csv file

In [65]:
csv_name = "reviews.csv"
review_df.to_csv(csv_name, index=False)